In [45]:
import sys
import math
import copy
import random
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt

from mesa import Agent
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.space import Grid
from mesa.time import RandomActivation
from mesa.batchrunner import BatchRunner

In [3]:
def get_datacollector(model):
    return model.datacollector


In [4]:
def get_agent_expiry_ticks(model):
    # In analysis will need to check expiry ticks column and 
    # cast to int to get summary stats.
    
    s = pd.DataFrame(columns=['unique_id', 'expiry_tick'])
    
    for a in model.schedule.agents:
        s = s.append({'unique_id': int(a.unique_id), 'expiry_tick': a.dead}, ignore_index=True)
    
    survival_rate = s['expiry_tick'].isna().sum() / len(s)
    
    s = s[['expiry_tick']].dropna().astype(int).describe()
    s.loc['survival_rate'] = survival_rate
    
    return s

In [5]:
def get_active_sensors_at_current_tick(model):
    return len([a for a in model.schedule.agents if np.isnan(a.dead)])

In [6]:
def get_total_data_collected(agent):
    return agent.db.sum()   

In [7]:
def get_informational_currency(agent):
    """
    Calculate the proportion of data collected by a 
    sensor that has been committed to the chain.
    
    **Note: This counts each tick as equal, even if 
    different volumes of data were collected during
    the ticks.**
    
    Attributes:
        agent: The Sensor for which to calculate 
            informational currency measure.
        window: The window through which to measure. 
        
            If window = 1, check proportion of data 
            reflected on chain from all time.
            
            If window > 1, check proportion of data in 
            most recent n=window ticks that is reflected 
            on chain
    """
    
    model = agent.model
    agent_id = agent.unique_id
    window = model.info_currency_window
    
    if window == 1 or window > len(model.blockchain.chain):
        try:
            return model.blockchain.chain[agent_id].value_counts()[True] / len(model.blockchain.chain[agent_id])
        except KeyError:
            return 0
    
    else:
        df = model.blockchain.chain[window * -1 :] # Last n rows
        try:
            return df[agent_id].value_counts()[True] / len(df[agent_id])
        except KeyError:
            return 0


In [134]:
class Sensor(Agent):
    """
    An edge device. 
    
    Attributes:
        battery_life: Total battery on board.
        mortal: a boolean value indicating whether the agent should
            cease activity once battery is depleted.
        record_cost, record_freq, record_bytes: Values
            to calculate sensor recording behavior
        compute_cost_per_byte: An abstracted link between
            bytes computed and energy consumed
        info_reduction: An abstracted scaling metric to 
            calculate the amount of data to transmit based
            on the amount of data collected at the edge. 
            Represents edge computing capability.
        sign_cost: An estimate of the energy consumed to sign a tx. 
        transmit_cost_per_byte: A link between bytes transmitted
            and energy consumed
        transmit_freq: How often data is transmitted from the sensor
            to the blockchain. If n > 1, every n ticks. If n < 1, 
            n is the probability of transmitting each tick
        gas_price: The gas_price a sensor is willing to pay to get 
            its transactions mined by the EVM.
        blockchain: a pointer to the blockchain each sensor is 
            connecting to, so it can invoke class methods
        stochasticity: The measure of randomness, applied when 
            appropriate, to simulate variation
        model: The model that is creating the sensor.
        
    """
    def __init__(self, unique_id, battery_life, mortal,
                record_cost, record_freq, record_bytes, 
                compute_cost_per_byte, info_reduction,
                sign_cost,
                transmit_cost_per_byte, transmit_freq,
                gas_price, blockchain, stochasticity,
                model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print('Creating Sensor agent ID', unique_id)
           
        self.battery_life = battery_life
        self.mortal = mortal
        self.dead = np.nan
        self.record_cost = record_cost
        self.record_freq = record_freq
        self.record_bytes = record_bytes
        self.compute_cost_per_byte = compute_cost_per_byte
        self.info_reduction = info_reduction
        self.sign_cost = sign_cost
        self.transmit_cost_per_byte = transmit_cost_per_byte
        self.transmit_freq = transmit_freq
        self.gas_price = gas_price
        self.blockchain = blockchain
        self.stochasticity = stochasticity
        
        self.gwei_spent = 0
        self.data_collected = 0
        self.last_sync = 0
        self.nonce = 0
        self.db = np.array([]) # << bytes recorded per tick
        
        self.blockchain.chain.loc[0, self.unique_id] = False

    
    def record(self):
        if (self.record_freq > 1 and self.model.schedule.steps % self.record_freq == 0) or self.record_freq > random.random():

                self.battery_life -= self.record_cost # << independent of number of bytes? 
                new_record = self.record_bytes
                if self.stochasticity:
                    new_record = math.ceil(new_record  * (1 + random.uniform(self.stochasticity * -1, self.stochasticity)))

                self.db = np.append(self.db, new_record)

        else:
            self.db = np.append(self.db, 0)

            
    def next_nonce(self):
        self.nonce += 1
        return self.nonce
   

    def transmit(self):
        
        if self.model.verbose:
            print("Transmitting tx from", self.unique_id)

        # Prepare data for transmission:
        
        # Calculate number of bytes to transmit (result of edge computation)
        bytes_collected = np.sum(self.db[self.last_sync : ])
        num_bytes_to_transmit = self.compute(bytes_collected)
        
        if num_bytes_to_transmit > 0:

            tx = self.create_tx_dict(self.next_nonce(), 
                           self.last_sync, self.model.schedule.steps, 
                           self.gas_price, num_bytes_to_transmit)

            # Prepare and sign tx
            self.sign()

            # Transmit, subtracting energy cost and adding gwei cost
            self.battery_life -= self.transmit_cost_per_byte * num_bytes_to_transmit
            self.blockchain.add_to_mempool(tx)

            self.last_sync = self.model.schedule.steps
    
    def create_tx_dict(self, nonce, start, end, gas_price, num_bytes):
        """
        Assemble variables into a dictionary representing 
        an (abstracted) Ethereum transaction.
        """

        tx = {}

        tx['nonce'] = nonce
        tx['from_address'] = self.unique_id
        tx['start_sync'] = start
        tx['end_sync'] = end
        tx['gas_price'] = gas_price
        # self.gas_limit = gas_limit # << unused parameter
        tx['num_bytes'] = num_bytes

        return tx
    
    def compute(self, num_bytes):
        
        # Only invoked from within the transmit() method
        
        if self.info_reduction is not 1:
            self.battery_life -= self.compute_cost_per_byte * num_bytes
            return math.ceil(self.info_reduction * num_bytes)
        else:
            return num_bytes
            # with no compute cost
    
    
    def sign(self):
        
        # Only invoked from within the transmit() method
        self.battery_life -= self.sign_cost
    
    
    def confirm_tx(self, tx):
        
        # Here we could include learning element ...

#         mine_lag =  self.model.schedule.steps - tx.block_submitted
        # if mine_lag took too long - or was too quick - according to some threshold:
        #      change transmit_freq, info_reduction or gas_price ...
        
        self.gwei_spent += tx.gas_spend
    
    
    def step(self):
        
        self.gwei_spent = 0
        
        if math.isnan(self.dead):   
            if self.battery_life < 0 and self.mortal:
                if self.model.verbose:
                    print("Sensor", self.unique_id, "out of battery at tick", self.model.schedule.steps)
                self.dead = self.model.schedule.steps

            self.record()    

            if self.transmit_freq >= 1:
                if self.model.schedule.steps % self.transmit_freq == 0:
                    self.transmit()
            elif self.transmit_freq > random.random():
                self.transmit()



In [98]:
class Blockchain(Agent):
    
    def __init__(self, unique_id, gas_price, block_gas_limit,
                gas_per_byte, gas_per_second, avg_block_time, model):
        
        super().__init__(unique_id, model)
        
        if self.model.verbose:
            print("Blockchain created: ID", unique_id)
        
        self.gas_price = gas_price
        self.block_gas_limit = block_gas_limit
        self.gas_per_byte = gas_per_byte
        self.gas_per_second = gas_per_second
        self.avg_block_time = avg_block_time
        self.chain = pd.DataFrame()
        
        self.tx_ct = 0
        self.mempool = pd.DataFrame(columns=["from_address", "nonce", 
                                             "start_sync", "end_sync", 
                                             "gas_price", "num_bytes", 
                                             "gas_spend", "tx_id",
                                             "mined", "block_submitted"])

    def add_to_mempool(self, tx):
        tx['gas_spend'] = tx['gas_price'] * self.gas_per_byte * tx['num_bytes']

        tx['tx_id'] = self.tx_ct
        tx['mined'] = False
        tx['block_submitted'] = self.model.schedule.steps
        row = pd.DataFrame(tx, index = [self.tx_ct])
        
        self.tx_ct += 1
        self.mempool = self.mempool.append(row, ignore_index=True)
    
    def write_data(self, num_bytes):

        gwei_spent = self.gas_per_byte * num_bytes * self.gas_price
        return gwei_spent

    def mine_block(self):
        
        if self.model.verbose:
            print("Mining BLOCK NUMBER:", self.model.schedule.steps)
        self.chain.loc[self.model.schedule.steps] = [False for col in self.chain.columns]

        # Sort mempool to get highest-value transactions
        mp = self.mempool[self.mempool['mined'] == False].sort_values(by=['gas_spend', 'block_submitted']).reset_index()

        if len(mp) > 0:
            
            mp['cum_gas'] = mp['gas_spend'].cumsum()     
            if mp['cum_gas'].max() > self.block_gas_limit:
                
                # If we cannot include all transactions in a block, fit as many as possible ...
                tx_mined = mp[0 : mp[mp['cum_gas'] > self.block_gas_limit].index[0]]
                
            else:
                tx_mined = mp[0 : ]
            
            if self.model.verbose:
                print("Mining", len(tx_mined), "out of", len(mp), "unvalidated transactions.\n")
                print("Gas value:", tx_mined['gas_spend'].sum(), '\n')
            
            for tx in tx_mined.iterrows():
                
                if self.model.verbose:
                    print("Mining tx id:", tx[1].tx_id)
                    
                self.model.schedule._agents[tx[1].from_address].confirm_tx(tx[1])
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "mined"] = True
                self.mempool.loc[self.mempool['tx_id'] == tx[1].tx_id, "block_mined"] = self.model.schedule.steps
                
                self.chain.loc[tx[1].start_sync : tx[1].end_sync, tx[1].from_address] = True
                
        else:
            if self.model.verbose:
                print('Empty mempool')
            pass
        
        if self.model.verbose:
            print('Block #', self.model.schedule.steps, 'mined.\n\n')
    
    # Not used:
    def compute(self, num_seconds):
        gwei_spent = self.gas_per_second * num_seconds * self.gas_price
        return gwei_spent
    

In [46]:
class SensorBlockchainNetwork(Model):
    
    def __init__(self, num_sensors=20,
                 stochasticity=0.05,
                 
       # Blockchain vars:
                 blockchain_gas_price=20,
                 block_gas_limit=9000000,
                 gas_per_byte=625,
                 gas_per_second=75000000,
                 avg_block_time=13,
                    # gas_per_byte and gas_per_second calculated based on 
                    # https://hackernoon.com/ether-purchase-power-df40a38c5a2f
                
        # Sensor vars:
                 battery_life=1000,
                 record_cost=1, record_freq=1, record_bytes=32,
                 compute_cost_per_byte=1, info_reduction=1,
                 sign_cost=0.1, 
                 transmit_cost_per_byte=1, transmit_freq=1,
                 sensor_gas_price=20,
                 mortal=True,
                 
                 # Model vars:
                 verbose=False,
                 info_currency_window=1
                ):
        
        super().__init__()
        
        
        self.verbose = verbose
        if self.verbose:
            print('Verbose model')
        self.info_currency_window = info_currency_window
        self.running = True
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
                                model_reporters = {
                                    "active_sensors": get_active_sensors_at_current_tick,
                                },
                                agent_reporters = {
                                    "gwei_spent": 'gwei_spent',
                                    "battery_life": 'battery_life',
                                    "data_collected": get_total_data_collected,
                                    "informational_currency": get_informational_currency
                                })
        
        self.blockchain = Blockchain(self.next_id(), blockchain_gas_price, block_gas_limit,
                                    gas_per_byte, gas_per_second, avg_block_time, self) 
        
        for i in range(num_sensors):
            sensor = Sensor(self.next_id(), battery_life, mortal,
                            record_cost, record_freq, record_bytes,
                            compute_cost_per_byte, info_reduction, 
                            sign_cost,
                            transmit_cost_per_byte, transmit_freq, 
                            sensor_gas_price,
                            self.blockchain, stochasticity, # << each sensor gets the same amount of stochasticity?
                            self)
            
            self.schedule.add(sensor)
    
        # Mine genesis block
        self.blockchain.chain.loc[1] = [False for col in self.blockchain.chain.columns]

        if self.verbose:
            print(num_sensors, "instantiated and added to schedule.")
    
    def step(self):
        self.schedule.step()
        if self.schedule.steps > 1:
            self.blockchain.mine_block()
        self.datacollector.collect(self)

In [ ]:
fixed_params = {
    "blockchain_gas_price": 20,
    "block_gas_limit": 8000000,
    "gas_per_byte": 625,
    "gas_per_second": 75000000,
    "avg_block_time": 13,
    "verbose": False,
    "battery_life": 10000,
    "mortal": False,
    "stochasticity": 0.05,
    "info_currency_window": 30,
    
    "num_sensors": 20,
    "record_bytes": 16,
    "record_freq": 0.3
}

In [145]:
model = SensorBlockchainNetwork(
    blockchain_gas_price= 20,
    block_gas_limit=8000000,
    gas_per_byte= 625,
    gas_per_second= 75000000,
    avg_block_time=13,
    verbose=False,
    battery_life=10000,
    mortal= True, 
    num_sensors=50,
    stochasticity=0.05,
    record_bytes=16, 
    info_currency_window=30)

In [146]:
for i in range(100):
    model.step()

In [149]:
model.blockchain.chain[-30:]

,2,3,4,5,6,7,8,9,10,11,...,42,43,44,45,46,47,48,49,50,51
71,False,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,True,False,False,False,False
72,False,False,False,True,False,True,True,False,False,False,...,True,False,True,False,False,True,False,False,False,False
73,True,True,True,True,False,True,False,True,False,False,...,True,False,False,False,False,False,True,False,True,False
74,True,True,True,False,False,False,True,True,False,False,...,False,False,False,False,False,False,True,False,True,False
75,True,False,True,False,False,True,True,False,False,False,...,False,False,False,True,False,False,False,False,False,True
76,True,True,False,True,True,True,False,True,False,True,...,False,False,False,True,False,False,False,True,False,True
77,False,True,True,True,True,True,False,True,False,True,...,False,False,False,True,False,True,True,True,False,False
78,False,False,True,True,False,True,False,False,False,False,...,False,False,False,True,False,True,True,True,False,False
79,True,False,True,True,False,True,False,False,False,False,...,False,True,False,True,False,True,True,False,False,True
80,True,False,True,False,True,True,False,False,False,False,...,False,True,False,True,False,True,False,True,False,True


# Model Execution and Analysis

Here we instantiate a mesa BatchRunner object, which contains methods for iterating through the arrays of variable parameters we pass in and executing a model for each arrangement. As a result we can access a pandas dataframe that contains all the agent, model tick and model run variables, as defined in the SensorBlockchainNetwork class definition, and in the model_reporters parameter passed into the BatchRunner object instantiation. 

We pass this dataframe into the following functions to extract relevant data and metrics, to be analyzed and visualized to discern system dynamic across the range of parameters swept. 

In [12]:
def extract_mining_summary_from_batch_df(batch_df):
    """
    A function extracting mining descriptive statistics
    from the dataframe returned from the 
    batchrunner.get_model_vars_dataframe() method invocation
    """
    df = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity', 'mean_mining_time', 'std', 'min', 'max'])

    for i, row in batch_df.iterrows():

        df.loc[i, 'num_sensors'] = row['num_sensors']
        df.loc[i, 'record_freq'] = row['record_freq']
        df.loc[i, 'record_bytes'] = row['record_bytes']
        df.loc[i, 'stochasticity'] = row['stochasticity']

        try:
            df.loc[i, 'mean_mining_time'] = row['mining_summary']['mean']
            df.loc[i, 'std'] = row['mining_summary']['std']
            df.loc[i, 'min'] = row['mining_summary']['min']
            df.loc[i, 'max'] = row['mining_summary']['max']
        except:
            df.loc[i, 'mean_mining_time'] = row['mining_summary']
    
    return df

In [13]:
def extract_gwei_spent_from_batch_df(batch_df):
    df_tmp = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity'])

    for i, row in batch_df.iterrows():
        df_tmp.loc[i, 'num_sensors'] = row['num_sensors']
        df_tmp.loc[i, 'record_freq'] = row['record_freq']
        df_tmp.loc[i, 'record_bytes'] = row['record_bytes']
        df_tmp.loc[i, 'stochasticity'] = row['stochasticity']

        df_gwei = row['data_collector'].get_agent_vars_dataframe()['gwei_spent'].unstack()

        for col in df_gwei.columns:
            df_gwei[str(col) + '_shift'] = df_gwei[col].shift(1)
            df_gwei[str(col) + '_gwei_spend'] = df_gwei[col] - df_gwei[str(col) + '_shift']

        df_gwei = df_gwei[[col for col in df_gwei.columns if 'spend' in str(col)]]

        df_tmp.loc[i, 'df_gwei_spent'] = InfoCurrency(df_gwei)

    return df_tmp

In [14]:
class InfoCurrency:
    """ A cheeky workaround to store dataframes as a value in another dataframe ..."""
    def __init__(self, df):
        self.df = df

def extract_informational_currency_from_batch_df(batch_df):
    
    df_tmp = pd.DataFrame(columns=['num_sensors',  'record_freq', 'record_bytes', 'stochasticity', 'info_currency_df'])
    for i, row in batch_df.iterrows():
        df_tmp.loc[i, 'num_sensors'] = row['num_sensors']
        df_tmp.loc[i, 'record_freq'] = row['record_freq']
        df_tmp.loc[i, 'record_bytes'] = row['record_bytes']
        df_tmp.loc[i, 'stochasticity'] = row['stochasticity']

        df_tmp.loc[i, 'info_currency_df'] = InfoCurrency(row['data_collector'].get_agent_vars_dataframe()['informational_currency'].unstack())
    
    return df_tmp

In [15]:
def extract_network_life_cycle_from_batch_df(batch_df):
    return batch_df[[ 'num_sensors','record_bytes', 'record_freq', 'stochasticity', 'agent_expiry_ticks']]

# Initial parameter sweep

Our initial batch run swept across several values in several dimensions. We collected a lot of data, and analyzed it to identify areas where a more specific sweep would be sensible. 

### Helper functions

Functions to serve as reporters, and one to help pickle the dataframe outputs of the batch run.

In [105]:
def pickle_batch_run_results(batch_run, outfile):
    
    model_dfs = batch_run.get_model_vars_dataframe()
    mining_cols = model_dfs['mining_summary'][0].index.tolist()
    
    for i, row in model_dfs[['mining_summary']].iterrows():
        for c in mining_cols:
            try:
                model_dfs.loc[i, 'mining_' + c] = row['mining_summary'][c]
            except:
                model_dfs.loc[i, 'mining_' + c] = np.nan

    
    d = {
        'batch_model_df': model_dfs,
        'batch_agent_df': batch_run.get_agent_vars_dataframe()
    }

    with open(outfile, 'wb') as f:
        pickle.dump(d, f)

In [17]:
def get_agent_vars_by_tick(model):
    return model.datacollector.get_agent_vars_dataframe()

In [18]:
def get_model_vars_by_tick(model):
    return model.datacollector.get_model_vars_dataframe()

In [43]:
def get_mining_summary(model):
    try:
        df = model.blockchain.mempool
        frac_unmined = df['block_mined'].isna().sum() / len(df)
        
        df = df.dropna()
        df['mining_time'] = df['block_mined'] - df['block_submitted']

        s = df['mining_time'].astype(int).describe() 
        s['frac_unmined'] = frac_unmined
        
        return s
        
    except:
        return "Failed on tick " + str(model.schedule.steps) + " --- " + sys.exc_info[0] 

## Sweeping Independent Variables

In [119]:
fixed_params = {
    "blockchain_gas_price": 20,
    "block_gas_limit": 8000000,
    "gas_per_byte": 625,
    "gas_per_second": 75000000,
    "avg_block_time": 13,
    "verbose": False,
    "battery_life": 10000,
    "mortal": False,
    "stochasticity": 0.05,
    "info_currency_window": 30,
    
    "num_sensors": 20,
    "record_bytes": 16,
    "record_freq": 0.3
}

In [150]:
np.linspace(0.9, 1.0, 10)

array([0.9       , 0.91111111, 0.92222222, 0.93333333, 0.94444444,
       0.95555556, 0.96666667, 0.97777778, 0.98888889, 1.        ])

In [ ]:
fixed_params_record_freq = copy.deepcopy(fixed_params)
del fixed_params_record_freq['record_freq']
fixed_params_record_freq['num_sensors'] = 40


variable_params_record_freq = {
        "record_freq": np.linspace(0.9, 1.0, 10)
#     "record_freq": [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
}

batch_runner_record_freq = BatchRunner(
    SensorBlockchainNetwork,
    variable_params_record_freq,
    fixed_params_record_freq,
    iterations = 3,
    max_steps = 300,
    model_reporters = {"agent_vars_by_tick": get_agent_vars_by_tick,
                   "model_vars_by_tick": get_model_vars_by_tick,
                   "mining_summary": get_mining_summary},
        
    agent_reporters = {"expiry_ticks": "dead"}

)

batch_runner_record_freq.run_all()
pickle_batch_run_results(batch_runner_record_freq, './data/run-4/batch_run_3_30_sensors_record_freq.pkl')

0it [00:00, ?it/s]/Users/iv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
20it [27:18, 81.04s/it]

In [142]:
fixed_params_num_sensors = copy.deepcopy(fixed_params)
del fixed_params_num_sensors['num_sensors']

variable_params_num_sensors = {
    "num_sensors": np.arange(0, 501, 50)[1:]
}

batch_runner_num_sensors = BatchRunner(
    SensorBlockchainNetwork,
    variable_params_num_sensors,
    fixed_params_num_sensors,
    iterations = 3,
    max_steps = 300,
    
    model_reporters = {"agent_vars_by_tick": get_agent_vars_by_tick,
                   "model_vars_by_tick": get_model_vars_by_tick,
                   "mining_summary": get_mining_summary},

    agent_reporters = {"expiry_ticks": "dead"}
)

batch_runner_num_sensors.run_all()
pickle_batch_run_results(batch_runner_num_sensors, './data/run-4/batch_run_5-iterations_num_sensors.pkl')

3it [03:02, 60.40s/it]/Users/iv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
30it [3:18:18, 816.55s/it]


In [144]:
fixed_params_record_bytes = copy.deepcopy(fixed_params)
del fixed_params_record_bytes['record_bytes']

variable_params_record_bytes = {
    "record_bytes": np.arange(1, 520, 20)
}

batch_runner_record_bytes = BatchRunner(
    SensorBlockchainNetwork,
    variable_params_record_bytes,
    fixed_params_record_bytes,
    iterations = 3,
    max_steps = 300,
    model_reporters = {"agent_vars_by_tick": get_agent_vars_by_tick,
                       "model_vars_by_tick": get_model_vars_by_tick,
                       "mining_summary": get_mining_summary},
    
    agent_reporters = {"expiry_ticks": "dead"}
)

batch_runner_record_bytes.run_all()
pickle_batch_run_results(batch_runner_record_bytes, './data/run-4/batch_run_1_record_bytes.pkl')

9it [03:35, 24.20s/it]/Users/iv/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
78it [22:03, 14.21s/it]
